# MERGE DNDS WITH TARGETS

Import packages:

In [1]:
import pandas as pd
import seaborn as sns

Import data:

In [2]:
targets_df =  pd.read_csv("../results/targets_from_xml_selected.csv", index_col=0)
dnds_df = pd.read_table("../data/ensembl/dn_ds_human_mouse_chimp.csv")
filenames_df = pd.read_csv("../data/ensembl/filenames_mapping.csv")

As many targets have non-unique UniProt IDs I decided to remove all ambiquous targets:

In [3]:
print("in total:", targets_df.shape[0], "targets")
print("without drugs:", sum(pd.isnull(targets_df['drugs_num'])), "targets")
print("without UniProt ID:", sum(targets_df['UniProt_ID'] == '-'), "targets")
print()
print("# IDs occurences - # of IDs (before filtering):")
print(targets_df['UniProt_ID'].value_counts().value_counts())

targets_df = targets_df[targets_df['UniProt_ID'] != '-']  # remove targets without UniProt IDs
targets_df.drop_duplicates(subset ="UniProt_ID", keep = False, inplace = True)  # remove targets with non-unique UniProt IDs

print()
print("# IDs occurences - # of IDs (after filtering):")
print(targets_df['UniProt_ID'].value_counts().value_counts())

targets_df.sort_values('UniProt_ID').head()

in total: 2555 targets
without drugs: 0 targets
without UniProt ID: 53 targets

# IDs occurences - # of IDs (before filtering):
1     2437
2       22
3        7
53       1
Name: UniProt_ID, dtype: int64

# IDs occurences - # of IDs (after filtering):
1    2437
Name: UniProt_ID, dtype: int64


,name,organism,GenBank_ID,UniProt_ID,drugs,drugs_num
BE0003619,Gamma-aminobutyric acid receptor subunit rho-3,Humans,AC026100,A8MPY1,DB00231; DB00546; DB00690; DB00801; DB00829; D...,15
BE0003600,BCR/ABL fusion protein isoform X9,Humans,EU216066,A9UF02,DB00619,1
BE0009155,Sn1-specific diacylglycerol lipase alpha,Humans,Y10601,F5GY58,DB14050,1
BE0004105,Serine/threonine-protein kinase Sgk1,Humans,Y10032,O00141,DB03247; DB07837; DB08191,3
BE0004639,"Thymidine kinase 2, mitochondrial",Humans,X56793,O00142,DB02452; DB02594; DB04485,3


Let's look at `filenames_df`. Some gene IDs are mapped to multiple UniProt IDs. I decided to leave it that way so we won't lose UniProt IDs when merging with `targets_df`:

In [4]:
print(filenames_df.shape)
print()

filenames_df.columns = ['Gene stable ID', 'UniProt_ID'] # rename the column so it can be used for merging

print("# IDs occurences - # of IDs:")
print(filenames_df['UniProt_ID'].value_counts().value_counts())

filenames_df.head()

(129449, 2)

# IDs occurences - # of IDs:
1     62479
2      3323
7       477
8       437
3       300
6       193
5       189
4       154
10       40
31       31
42       28
36       24
14       17
41       17
33        9
28        5
9         4
39        3
40        3
13        3
12        2
15        2
35        2
30        1
Name: UniProt_ID, dtype: int64


,Gene stable ID,UniProt_ID
0,ENSG00000210049,NaN
1,ENSG00000211459,NaN
2,ENSG00000210077,NaN
3,ENSG00000210082,NaN
4,ENSG00000209082,NaN


Merge `targets_df` with `filenames_df` first so every target can be then mapped to its gene in `dnds_df`. One should not merge `filenames_df` with `dnds_df` first as `filenames_df` includes multiple UniProt IDs for each gene. Merging with `targets_df` leaves only needed IDs present in `targets_df`.

In [5]:
targets_mapped_df = pd.merge(targets_df, filenames_df, how='left', on='UniProt_ID')
print(targets_mapped_df.shape)
print(sum(pd.isnull(targets_mapped_df['Gene stable ID'])), "targets unmapped")
targets_mapped_df.head()

(2698, 7)
92 targets unmapped


,name,organism,GenBank_ID,UniProt_ID,drugs,drugs_num,Gene stable ID
0,"Alanine--glyoxylate aminotransferase 2, mitoch...",Humans,AJ292204,Q9BYV1,DB00114,1,ENSG00000113492
1,Glutamate decarboxylase 1,Humans,M81883,Q99259,DB00114; DB00142,2,ENSG00000128683
2,Cystathionine beta-synthase,Humans,L19501,P35520,DB00114; DB00118; DB00133; DB00151,4,ENSG00000160200
3,Kynureninase,Humans,U57721,Q16719,DB00114; DB07069,2,ENSG00000115919
4,"Serine hydroxymethyltransferase, cytosolic",Humans,L11931,P34896,DB00114; DB00116; DB01055; DB02067; DB02800; D...,6,ENSG00000176974


Let's now look at `dnds_df`. How many Human genes do have both orthologs and calculated dN, dS?

In [7]:
print("with ortholog (for Chimpanzee):", sum(pd.notnull(dnds_df['Chimpanzee gene name'])), "genes")
print("with dN/dS (for Chimpanzee):", sum(pd.notnull(dnds_df['dN with Chimpanzee'])), "genes")
print("with ortholog (for Mouse):", sum(pd.notnull(dnds_df['Mouse gene name'])), "genes")
print("with dN/dS (for Mouse):", sum(pd.notnull(dnds_df['dN with Mouse'])), "genes")
print("with both orthologs:", sum(pd.notnull(dnds_df['Mouse gene name']) & pd.notnull(dnds_df['Chimpanzee gene name'])), "genes")
print("with both dN/dS:", sum(pd.notnull(dnds_df['dN with Mouse']) & pd.notnull(dnds_df['dN with Chimpanzee'])), "genes")

print("in total:", dnds_df.shape[0], "genes")

dnds_df.head()

with ortholog (for Chimpanzee): 25484 genes
with dN/dS (for Chimpanzee): 25057 genes
with ortholog (for Mouse): 29290 genes
with dN/dS (for Mouse): 25649 genes
with both orthologs: 20522 genes
with both dN/dS: 22603 genes
in total: 77255 genes


,Gene stable ID,Gene name,Gene description,Chimpanzee gene name,dN with Chimpanzee,dS with Chimpanzee,Mouse gene name,dN with Mouse,dS with Mouse
0,ENSG00000210049,MT-TF,mitochondrially encoded tRNA-Phe (UUU/C) [Sour...,NaN,NaN,NaN,NaN,NaN,NaN
1,ENSG00000211459,MT-RNR1,mitochondrially encoded 12S rRNA [Source:HGNC ...,NaN,NaN,NaN,NaN,NaN,NaN
2,ENSG00000210077,MT-TV,mitochondrially encoded tRNA-Val (GUN) [Source...,NaN,NaN,NaN,NaN,NaN,NaN
3,ENSG00000210082,MT-RNR2,mitochondrially encoded 16S rRNA [Source:HGNC ...,NaN,NaN,NaN,NaN,NaN,NaN
4,ENSG00000209082,MT-TL1,mitochondrially encoded tRNA-Leu (UUA/G) 1 [So...,NaN,NaN,NaN,NaN,NaN,NaN


How many genes have 1:1 orthologs?

In [28]:
print("# of human genes - # of orthologs")
dnds_df['Gene stable ID'].value_counts(dropna=False).value_counts()

# of human genes - # of orthologs


1      64458
2       1810
3        300
4        166
5         88
6         52
7         33
14        26
9         22
8         20
93        13
58        12
10        10
186        9
15         8
12         6
110        6
29         6
19         5
28         5
17         4
13         4
11         3
16         3
116        3
27         3
42         2
20         2
25         2
54         1
21         1
26         1
23         1
22         1
18         1
Name: Gene stable ID, dtype: int64

Leave only genes with 1:1 orthologs present both in Chimpanzee and Mouse:

In [29]:
print('before filtering:', dnds_df.shape[0], 'genes')
print()

dnds_sel_df = dnds_df.dropna(subset=['dN with Mouse','dN with Chimpanzee','dS with Mouse','dS with Chimpanzee'])
print('with orthologs in both organisms:', dnds_sel_df.shape[0], 'genes')
print()

dnds_sel_df = dnds_sel_df.drop_duplicates(subset = ["Gene stable ID"], keep = False).reset_index(drop=True)
print('with 1:1 orthologs only:', dnds_sel_df.shape[0], 'genes')

dnds_sel_df.head()

before filtering: 77255 genes

with orthologs in both organisms: 22603 genes

with 1:1 orthologs only: 14903 genes


,Gene stable ID,Gene name,Gene description,Chimpanzee gene name,dN with Chimpanzee,dS with Chimpanzee,Mouse gene name,dN with Mouse,dS with Mouse
0,ENSG00000228253,MT-ATP8,mitochondrially encoded ATP synthase membrane ...,MT-ATP8,0.0325,0.3331,mt-Atp8,0.4871,0.8480
1,ENSG00000198695,MT-ND6,mitochondrially encoded NADH:ubiquinone oxidor...,MT-ND6,0.0204,0.6559,mt-Nd6,0.3455,13.3498
2,ENSG00000169926,KLF13,Kruppel like factor 13 [Source:HGNC Symbol;Acc...,KLF13,0.0026,0.0771,Klf13,0.0219,0.6810
3,ENSG00000276289,KCNE1B,potassium voltage-gated channel subfamily E re...,NaN,0.0165,0.0649,Kcne1,0.1382,0.8135
4,ENSG00000067992,PDK3,pyruvate dehydrogenase kinase 3 [Source:HGNC S...,PDK3,0.0152,0.0300,Pdk3,0.0151,0.5213


Create `grand_df` which is `dnds_sel_df` merged with `targets_mapped_df`:

In [30]:
grand_df = pd.merge(dnds_sel_df, targets_mapped_df, how='left', on='Gene stable ID')
print(grand_df.shape)
grand_df.head()

(14907, 15)


,Gene stable ID,Gene name,Gene description,Chimpanzee gene name,dN with Chimpanzee,dS with Chimpanzee,Mouse gene name,dN with Mouse,dS with Mouse,name,organism,GenBank_ID,UniProt_ID,drugs,drugs_num
0,ENSG00000228253,MT-ATP8,mitochondrially encoded ATP synthase membrane ...,MT-ATP8,0.0325,0.3331,mt-Atp8,0.4871,0.8480,NaN,NaN,NaN,NaN,NaN,NaN
1,ENSG00000198695,MT-ND6,mitochondrially encoded NADH:ubiquinone oxidor...,MT-ND6,0.0204,0.6559,mt-Nd6,0.3455,13.3498,NADH-ubiquinone oxidoreductase chain 6,Humans,J01415,P03923,DB00157,1.0
2,ENSG00000169926,KLF13,Kruppel like factor 13 [Source:HGNC Symbol;Acc...,KLF13,0.0026,0.0771,Klf13,0.0219,0.6810,NaN,NaN,NaN,NaN,NaN,NaN
3,ENSG00000276289,KCNE1B,potassium voltage-gated channel subfamily E re...,NaN,0.0165,0.0649,Kcne1,0.1382,0.8135,NaN,NaN,NaN,NaN,NaN,NaN
4,ENSG00000067992,PDK3,pyruvate dehydrogenase kinase 3 [Source:HGNC S...,PDK3,0.0152,0.0300,Pdk3,0.0151,0.5213,[Pyruvate dehydrogenase [lipoamide]] kinase is...,Humans,L42452,Q15120,DB03758; DB03760,2.0


Add columns with dN/dS ratios and whether the gene is present in `targets_mapped_df` or not:

In [32]:
grand_df['dN/dS with Chimpanzee'] = round(grand_df['dN with Chimpanzee'] / grand_df['dS with Chimpanzee'], 4)
grand_df['dN/dS with Mouse'] = round(grand_df['dN with Mouse'] / grand_df['dS with Mouse'], 4)
grand_df['present in DrugBank'] = pd.notna(grand_df['drugs_num'])
grand_df.head()

,Gene stable ID,Gene name,Gene description,Chimpanzee gene name,dN with Chimpanzee,dS with Chimpanzee,Mouse gene name,dN with Mouse,dS with Mouse,name,organism,GenBank_ID,UniProt_ID,drugs,drugs_num,dN/dS with Chimpanzee,dN/dS with Mouse,present in DrugBank
0,ENSG00000228253,MT-ATP8,mitochondrially encoded ATP synthase membrane ...,MT-ATP8,0.0325,0.3331,mt-Atp8,0.4871,0.8480,NaN,NaN,NaN,NaN,NaN,NaN,0.0976,0.5744,False
1,ENSG00000198695,MT-ND6,mitochondrially encoded NADH:ubiquinone oxidor...,MT-ND6,0.0204,0.6559,mt-Nd6,0.3455,13.3498,NADH-ubiquinone oxidoreductase chain 6,Humans,J01415,P03923,DB00157,1.0,0.0311,0.0259,True
2,ENSG00000169926,KLF13,Kruppel like factor 13 [Source:HGNC Symbol;Acc...,KLF13,0.0026,0.0771,Klf13,0.0219,0.6810,NaN,NaN,NaN,NaN,NaN,NaN,0.0337,0.0322,False
3,ENSG00000276289,KCNE1B,potassium voltage-gated channel subfamily E re...,NaN,0.0165,0.0649,Kcne1,0.1382,0.8135,NaN,NaN,NaN,NaN,NaN,NaN,0.2542,0.1699,False
4,ENSG00000067992,PDK3,pyruvate dehydrogenase kinase 3 [Source:HGNC S...,PDK3,0.0152,0.0300,Pdk3,0.0151,0.5213,[Pyruvate dehydrogenase [lipoamide]] kinase is...,Humans,L42452,Q15120,DB03758; DB03760,2.0,0.5067,0.0290,True


Some genes have both dN and dS equal to 0. For these genes dN/dS is set to 'NaN':

In [59]:
print(sum(pd.isna(grand_df['dN/dS with Chimpanzee'])), 'genes with dN/dS = NaN (Chimpanzee)')
#print(sum((grand_df['dN with Chimpanzee'] == 0) & (grand_df['dS with Chimpanzee'] == 0)))

print(sum(pd.isna(grand_df['dN/dS with Mouse'])), 'genes with dN/dS = NaN (Mouse)')
#print(sum((grand_df['dN with Mouse'] == 0) & (grand_df['dS with Mouse'] == 0)))

2971 genes with dN/dS = NaN (Chimpanzee)
256 genes with dN/dS = NaN (Mouse)


Export `grand_df` to a `.csv` file:

In [33]:
grand_df[['Gene name', 'Gene description', 'dN/dS with Chimpanzee', 'dN/dS with Mouse', 'present in DrugBank']].to_csv('../results/dnds_for_targets.csv')
